#### Milestone II
#### Michael Perrine
#### DSC 540
#### Professor Williams


<h1><center>Milestone 1</center></h1>

# <h1><center>Market Analysis</center></h1>


The purpose of this analysis is to research market data. I will gather data from several sources. The goal is to analyze the effect of economic data on stock performance. I will also use the S&P 500 as the benchmark for my stock performance. The sources of my data will come from various sources. I will pull the stock data from FMP a stock price API. The web address for FMP is [Documentation V2 - API Reference | FMP](https://site.financialmodelingprep.com/developer/docs).  The economic data will come from the FRED. The FRED is managed by the St. louis Fed. The web address is Federal Reserve Economic Data | FRED | St. Louis Fed. Finally,  I will pull S&P data from Wikipedia. The web address is S&P 500 - Wikipedia.  Economic data will show a relationship between the stock data and the economy. And the S&P will show the relationship between the stock index and the market. 
	
The plan for my analysis will be to add the different data sets into one data frame. I will run an exploratory analysis on the data. I will need to transform the data and determine the variables that make the greatest impact on the analysis. It will be important for me to build graphs to determine the connection between the variables. After I build my charts, I can  run a regression analysis and test my model. For the regression analysis I will need to split my data into a training and a testing set. Along with any analysis there are challenges that I will have to manage. 

Some challenges that I can foresee are issues with combining the different data sets. Since I am working with data sets from multiple sources, I’ll have to make sure they are formatted appropriately. There may also be problems with loading the data into the program. The data will be stored in different formats, so I’ll have to ensure that they are formatted in a manner that is compatible across data sets. Another challenge that I can anticipate is overfitting the data. I need to make that the training and testing sets are not in danger of overfitting the data.

Ethical concerns are always a factor when manipulating data. It is important to implement safeguards to prevent ethical. One ethical concern is the  misrepresentation of the data. It is important that I don’t allow the data to misrepresent the results. Another ethical issue is bias. It’s important that I don’t allow my personal bias to cause me to see relationships that don’t exist. 
	


<h1><center>Milestone 2</center></h1>

The first series of code will load all the libraries for the analysis

In [89]:
# Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
import yfinance as yf
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests 
import lxml
import warnings
import re
import csv
from financetoolkit import Toolkit
from dotenv import load_dotenv
import os

In [2]:
warnings.filterwarnings('ignore')

1. Prep the flat files


The next series of code loads the data from the FRED. I will use three data sets for my analysis.
The first data set is the Coincident Economic Activity Index. This index measures the US economic condition. It takes into account non-farm payroll employment, the unemployment rate, average hours worked in manufacturing, and wages and salaries. The second data set is the Consumer Price Index. This measures the price changes of goods and services and is a leading indicator of inflation. The final data set from the FRED is the One Year T-Bill. Fluctuations in T-Bill rates indicate direction of the US Monetary Policy.

In [3]:
# This code loads in the cea data, creates a pandas dataframe, and displays the first 5 rows
cea = pd.read_csv(r"coincident economic activity index.csv")
cea = pd.DataFrame(cea)
cea.head()

,observation_date,USPHCI
0,1/1/1979,45.36
1,2/1/1979,45.51
2,3/1/1979,45.75
3,4/1/1979,45.82
4,5/1/1979,46.05


In [4]:
# This code loads in the cpi data , creates a pandas dataframe, and displays the first 5 rows
cpi = pd.read_csv(r"monthly cpi.csv")
cpi = pd.DataFrame(cpi)
cpi.head()

,observation_date,CPIAUCSL
0,1/1/1947,21.48
1,2/1/1947,21.62
2,3/1/1947,22.00
3,4/1/1947,22.00
4,5/1/1947,21.95


In [5]:
# This code loads in the t-bill data , creates a pandas dataframe,and displays the first 5 rows
t_bill = pd.read_csv(r"one year t-bill.csv")
t_bill = pd.DataFrame(t_bill)
t_bill.head()

,observation_date,TB1YR
0,7/1/1959,4.34
1,8/1/1959,4.31
2,9/1/1959,4.83
3,10/1/1959,4.69
4,11/1/1959,4.54


The next step is to combine the three data frames into one. This will give me the opportunity to do further analysis. The following series of codes will prep and combine the three data frames.

In [6]:
# The first line of code will store all three data frames into one object
dfs = (cea, cpi, t_bill)

In [7]:
# The second line of code combines all the data into one new data frame.
new_data = reduce(lambda left, right: pd.merge(left, right, on = "observation_date", how = "outer"),dfs)


In [8]:
# The third line of code renames the columns in the data frame and displays the completed data frame
new_data = new_data.set_axis(["date", "cea", "cpi" , "t_bill"], axis=1)
new_data

,date,cea,cpi,t_bill
0,1/1/1947,NaN,21.480,NaN
1,1/1/1948,NaN,23.680,NaN
2,1/1/1949,NaN,24.010,NaN
3,1/1/1950,NaN,23.510,NaN
4,1/1/1951,NaN,25.380,NaN
...,...,...,...,...
934,9/1/2020,122.45,259.997,0.13
935,9/1/2021,130.21,273.942,0.07
936,9/1/2022,136.71,296.421,3.73
937,9/1/2023,140.69,307.374,5.15


Now that I have one data frame I want to reverse the order of the elements. Currently they are in ascending order. I want them in descending order that way when I add my stock data I'll see everything from the most current date to the oldest date.

In [9]:
# This line of code reverses the order of the elements
new_data = new_data.iloc[::-1].reset_index(drop=True)
new_data


,date,cea,cpi,t_bill
0,9/1/2024,144.20,314.851,3.87
1,9/1/2023,140.69,307.374,5.15
2,9/1/2022,136.71,296.421,3.73
3,9/1/2021,130.21,273.942,0.07
4,9/1/2020,122.45,259.997,0.13
...,...,...,...,...
934,1/1/1951,NaN,25.380,NaN
935,1/1/1950,NaN,23.510,NaN
936,1/1/1949,NaN,24.010,NaN
937,1/1/1948,NaN,23.680,NaN


Now I want to view the data types in my data frame. It's important to make sure all the data types are appropriate for the elements. It looks like I need to change the date data type. It's listed as an object and should be listed as date/time.

In [10]:
# This code displays the data type
new_data.dtypes

date       object
cea       float64
cpi       float64
t_bill    float64
dtype: object

In [11]:
new_data.head()

,date,cea,cpi,t_bill
0,9/1/2024,144.20,314.851,3.87
1,9/1/2023,140.69,307.374,5.15
2,9/1/2022,136.71,296.421,3.73
3,9/1/2021,130.21,273.942,0.07
4,9/1/2020,122.45,259.997,0.13


In [12]:
# This code changes the date data type from object to date/time and displays the result
new_data["date"] = pd.to_datetime(new_data["date"],format='mixed')
new_data.dtypes

date      datetime64[ns]
cea              float64
cpi              float64
t_bill           float64
dtype: object

In [13]:
# This code displays the new date format
new_data.head()

,date,cea,cpi,t_bill
0,2024-09-01,144.20,314.851,3.87
1,2023-09-01,140.69,307.374,5.15
2,2022-09-01,136.71,296.421,3.73
3,2021-09-01,130.21,273.942,0.07
4,2020-09-01,122.45,259.997,0.13


In [14]:
# This code drops NaN values. I chose to drop the 
new_data_1 = new_data.dropna(how="any")
new_data_1

,date,cea,cpi,t_bill
0,2024-09-01,144.20,314.851,3.87
1,2023-09-01,140.69,307.374,5.15
2,2022-09-01,136.71,296.421,3.73
3,2021-09-01,130.21,273.942,0.07
4,2020-09-01,122.45,259.997,0.13
...,...,...,...,...
902,1983-01-01,47.12,97.900,8.01
903,1982-01-01,47.54,94.400,12.77
904,1981-01-01,47.16,87.200,12.62
905,1980-01-01,46.74,78.000,10.96


In [15]:
# This code validates the dimensions of the cleaned data frame
new_data_1.shape

(475, 4)

<h1><center>Milestone 3</center></h1>

In this section I am scraping data from Wikipedia. My goal is to extract the S&P500 table. To accomplish this goal I will use beautiful soup and requests to import the data. I will need to find the right html tags to create the desired table. The issue with source code is they are not always pretty or easy to work with. Challenges that I face are finding the right tags and dealing with missing or incomplete data.

In [16]:
# This code creates a url object
url = 'https://en.wikipedia.org/wiki/S%26P_500'

In [17]:
# This code requests data from the website
data = requests.get(url)

In [18]:
# This code extracts the html from wikipedia and displays it
soup = BeautifulSoup(data.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   S&amp;P 500 - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-ena

In [19]:
# This code locates the desired table
table = soup.find_all('table')[1]
table

<table class="wikitable mw-collapsible sortable" style="text-align:right;">
<caption class="nowrap">Show / Hide table
</caption>
<tbody><tr>
<th rowspan="2">Year
</th>
<th rowspan="2">Change in<br/>Index
</th>
<th rowspan="2">Total<br/>Annual Return,<br/>including<br/>dividends
</th>
<th rowspan="2">Value of $1.00<br/>invested on<br/>January 1, 1970
</th>
<th colspan="5">Annualized Return over
</th></tr>
<tr>
<th>5 years
</th>
<th>10 years
</th>
<th>15 years
</th>
<th>20 years
</th>
<th>25 years
</th></tr>
<tr>
<td>1961
</td>
<td>23.13%
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td></tr>
<tr>
<td>1962
</td>
<td style="color:red">-11.81%
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td></tr>
<tr>
<td>1963
</td>
<td>18.89%
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td></tr>
<tr>
<td>1964
</td>
<td>12.97%
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-


In [20]:
# This code parses the columns for the dataframe
data_columns = soup.find_all('th')[10:20]
data_columns

[<th rowspan="2">Year
 </th>,
 <th rowspan="2">Change in<br/>Index
 </th>,
 <th rowspan="2">Total<br/>Annual Return,<br/>including<br/>dividends
 </th>,
 <th rowspan="2">Value of $1.00<br/>invested on<br/>January 1, 1970
 </th>,
 <th colspan="5">Annualized Return over
 </th>,
 <th>5 years
 </th>,
 <th>10 years
 </th>,
 <th>15 years
 </th>,
 <th>20 years
 </th>,
 <th>25 years
 </th>]

In [21]:
# This code cleans the column titles
data_columns_titles = [title.text.strip() for title in data_columns]
data_columns_titles

['Year',
 'Change inIndex',
 'TotalAnnual Return,includingdividends',
 'Value of $1.00invested onJanuary 1, 1970',
 'Annualized Return over',
 '5 years',
 '10 years',
 '15 years',
 '20 years',
 '25 years']

In [22]:
# This code creates my empty data frame
s_p_index = pd.DataFrame(columns=[data_columns_titles])

In [23]:
# I removed the annualized return over column because it is not needed for my analysis
s_p_index.drop(columns=['Annualized Return over'], inplace=True)

In [24]:
# This code displays the empty dataframe
s_p_index

,Year,Change inIndex,"TotalAnnual Return,includingdividends","Value of $1.00invested onJanuary 1, 1970",5 years,10 years,15 years,20 years,25 years


In [25]:
# This code renames the columns
s_p_index.rename(columns={'Year':'year', 'Change inIndex':'change_in_index', 'TotalAnnual Return,includingdividends':'tot_ann_ret',
                'Value of $1.00invested onJanuary 1, 1970':'Value_of_1_invested_on_jan_1_1970',
                '5 years':'5_years', '10 years':'10_years', '15 years':'15_years','20 years':'20_years',
                '25 years':'25_years'}, inplace = True)

In [26]:
# this code displays the renamed columns
s_p_index

,year,change_in_index,tot_ann_ret,Value_of_1_invested_on_jan_1_1970,5_years,10_years,15_years,20_years,25_years


In [27]:
# This code locates all the row data for my S&P 500 table
column_data = table.find_all('tr')
column_data

[<tr>
 <th rowspan="2">Year
 </th>
 <th rowspan="2">Change in<br/>Index
 </th>
 <th rowspan="2">Total<br/>Annual Return,<br/>including<br/>dividends
 </th>
 <th rowspan="2">Value of $1.00<br/>invested on<br/>January 1, 1970
 </th>
 <th colspan="5">Annualized Return over
 </th></tr>,
 <tr>
 <th>5 years
 </th>
 <th>10 years
 </th>
 <th>15 years
 </th>
 <th>20 years
 </th>
 <th>25 years
 </th></tr>,
 <tr>
 <td>1961
 </td>
 <td>23.13%
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td></tr>,
 <tr>
 <td>1962
 </td>
 <td style="color:red">-11.81%
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td></tr>,
 <tr>
 <td>1963
 </td>
 <td>18.89%
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td></tr>,
 <tr>
 <td>1964
 </td>
 <td>12.97%
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td>
 <td>-
 </td></tr

In [28]:
# This code iterates through the rows to find the row data
for row in column_data:
    row_data = row.find_all('td')
    
    ind_row_data = [data.text.strip() for data in row_data]
    print(ind_row_data)

[]
[]
['1961', '23.13%', '-', '-', '-', '-', '-', '-', '-']
['1962', '-11.81%', '-', '-', '-', '-', '-', '-', '-']
['1963', '18.89%', '-', '-', '-', '-', '-', '-', '-']
['1964', '12.97%', '-', '-', '-', '-', '-', '-', '-']
['1965', '9.06%', '-', '-', '-', '-', '-', '-', '-']
['1966', '-13.09%', '-', '-', '-', '-', '-', '-', '-']
['1967', '20.09%', '-', '-', '-', '-', '-', '-', '-']
['1968', '7.66%', '-', '-', '-', '-', '-', '-', '-']
['1969', '-11.36%', '-', '-', '-', '-', '-', '-', '-']
['1970', '0.10%', '4.01%', '$1.04', '-', '-', '-', '-', '-']
['1971', '10.79%', '14.31%', '$1.19', '-', '-', '-', '-', '-']
['1972', '15.63%', '18.98%', '$1.41', '-', '-', '-', '-', '-']
['1973', '−17.37%', '−14.66%', '$1.21', '-', '-', '-', '-', '-']
['1974', '−29.72%', '−26.47%', '$0.89', '−2.35%', '-', '-', '-', '-']
['1975', '31.55%', '37.20%', '$1.22', '3.21%', '-', '-', '-', '-']
['1976', '19.15%', '23.84%', '$1.51', '4.87%', '-', '-', '-', '-']
['1977', '−11.50%', '−7.18%', '$1.40', '−0.21%', '-

After locating all the data for my analysis, I ran into a snag. I have empty lists in my data and was unable to append it to my empty data frame that I created earlier in the analysis. I tried to find a way to remove the empty lists and was unsuccessful. My solution is to append the list to a csv file and import the file to continue my preprocessing.

In [29]:
# This code displays a list of my columns from my s_p_index dataframe
s_p_index.columns

MultiIndex([(                             'year',),
            (                  'change_in_index',),
            (                      'tot_ann_ret',),
            ('Value_of_1_invested_on_jan_1_1970',),
            (                          '5_years',),
            (                         '10_years',),
            (                         '15_years',),
            (                         '20_years',),
            (                         '25_years',)],
           )

In [30]:
# This series of code creates my csv file and imports the data into the file
csv_file= open('s_p_scrape.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['year','change_in_index',  'tot_ann_ret', 'Value_of_1_invested_on_jan_1_1970',
                    '5_years', '10_years', '15_years', '20_years', '25_years'])
for row in column_data[2:]:
    row_data = row.find_all('td')
    
    ind_row_data = [data.text.strip() for data in row_data]
    #print(ind_row_data)
    csv_writer.writerow(ind_row_data)
csv_file.close()


In [31]:
# This code imports the csv file previously created
gspc = pd.read_csv(r's_p_scrape.csv')

In [32]:
# This code creates a dataframe for the csv
gspc=pd.DataFrame(gspc)
gspc.head(5)

,year,change_in_index,tot_ann_ret,Value_of_1_invested_on_jan_1_1970,5_years,10_years,15_years,20_years,25_years
0,1961,23.13%,-,-,-,-,-,-,-
1,1962,-11.81%,-,-,-,-,-,-,-
2,1963,18.89%,-,-,-,-,-,-,-
3,1964,12.97%,-,-,-,-,-,-,-
4,1965,9.06%,-,-,-,-,-,-,-


In [33]:
# This code clears the duplicate rows in the data frame
gspc_cleaned = gspc.drop_duplicates()
gspc_cleaned.head()


,year,change_in_index,tot_ann_ret,Value_of_1_invested_on_jan_1_1970,5_years,10_years,15_years,20_years,25_years
0,1961,23.13%,-,-,-,-,-,-,-
1,1962,-11.81%,-,-,-,-,-,-,-
2,1963,18.89%,-,-,-,-,-,-,-
3,1964,12.97%,-,-,-,-,-,-,-
4,1965,9.06%,-,-,-,-,-,-,-


In [34]:
# This code drops the first index in the data frame
gspc_cleaned.drop(gspc_cleaned.index[0], inplace= True)

In [35]:
# This code shows the cleaned data ready for analysis
gspc_cleaned.head()


,year,change_in_index,tot_ann_ret,Value_of_1_invested_on_jan_1_1970,5_years,10_years,15_years,20_years,25_years
1,1962,-11.81%,-,-,-,-,-,-,-
2,1963,18.89%,-,-,-,-,-,-,-
3,1964,12.97%,-,-,-,-,-,-,-
4,1965,9.06%,-,-,-,-,-,-,-
5,1966,-13.09%,-,-,-,-,-,-,-


<h1><center>Milestone 4</center></h1>

In [ ]:
# This code creates load_dotenv
load_dotenv()

True

In [ ]:
# This code retrieves api key
API_KEY=os.getenv("API_KEY")

In [ ]:
# This code creates the list of companies for my stock data
companies = Toolkit(['AAPL', 'MSFT', 'NVDA', 'JNJ', 'PG', 'JPM'], api_key=API_KEY, 
                    start_date= '2020-01-01')

In [ ]:
# This code displays all columns in the data frame
pd.set_option('display.max_columns', None)

In [ ]:
# This code retrieves the stock data from the FMP api
historical_data = companies.get_historical_data(period='monthly')
historical_data

Obtaining historical data: 100%|██████████| 7/7 [00:00<00:00,  8.93it/s]


Open                                                High         \
          AAPL   MSFT   NVDA    JNJ     PG    JPM Benchmark   AAPL   MSFT   
Date                                                                        
2020-01  80.23 172.21    6.1 149.75 125.99 134.49     327.0  80.67  172.4   
2020-02  64.32 152.41   6.06 135.36 109.46 117.52     288.7   69.6 163.71   
2020-03   63.9  159.4   6.68 134.99  113.5  92.05    260.56  65.62 164.78   
2020-04  72.49  180.0   7.41 148.46 115.92   95.6    291.71  73.63  180.4   
2020-05  79.81 182.73   8.55 147.96 116.13  98.66    302.46  80.29 184.27   
...        ...    ...    ...    ...    ...    ...       ...    ...    ...   
2025-01 247.19 418.98 123.78 152.62  166.0 269.23     607.5 247.19 420.69   
2025-02 236.95 392.66 118.02 164.31 173.58 260.73    585.56 242.09 397.63   
2025-03 217.01 372.54 105.13 164.37  168.1 239.41    549.83 225.62 377.07   
2025-04  209.3  390.3 104.47  158.2 162.45 241.21    547.57 213.58 396.66   
2025-05 212.36 452.05 136.22 149.67 162.65  267.5    591.25 212.57 454.36   

                                                 Low                       \
          NVDA    JNJ     PG    JPM Benchmark   AAPL   MSFT   NVDA    JNJ   
Date                                                                        
2020-01   6.11  150.0 126.95 134.83    327.17  77.07 169.58   5.87 148.64   
2020-02   6.81 137.29 113.23 118.49    297.89  64.09  152.0   6.04 130.82   
2020-03   6.88 135.71 114.29  94.17    263.33   63.0 156.56   6.44 130.05   
2020-04   7.46 150.43 118.28  96.84    293.32  72.09 176.23   7.29 147.16   
2020-05   8.88 149.35 116.72  99.18    304.96  79.12 180.41   8.49 145.33   
...        ...    ...    ...    ...       ...    ...    ...    ...    ...   
2025-01 127.85 153.18 166.62 270.82    609.96 233.44 414.91 119.19 152.06   
2025-02 125.09 165.94 174.65 264.81    594.72  230.2 386.57  116.4  162.7   
2025-03 110.96 166.63 171.14 245.86    560.71 216.23 367.24 103.65 164.23   
2025-04 108.92  158.2 163.16 245.22    556.52 206.67 384.44 104.08 155.02   
2025-05 136.35  151.5 163.43 268.46     594.5 209.77 448.73 133.46 149.22   

                                 Close                                     \
            PG    JPM Benchmark   AAPL   MSFT   NVDA    JNJ     PG    JPM   
Date                                                                        
2020-01 124.46 131.47    320.73  77.38 170.23   5.91 148.87 124.62 132.36   
2020-02 106.67 112.66    285.54  68.34 162.01   6.75 134.48 113.23 116.11   
2020-03 108.95  89.34    256.22  63.57 157.71   6.59 131.13  110.0  90.03   
2020-04 115.66  94.55    288.59  73.45 179.21   7.31 150.04 117.87  95.76   
2020-05 114.62  96.78    299.47  79.49 183.25   8.88 148.75 115.92  97.31   
...        ...    ...       ...    ...    ...    ...    ...    ...    ...   
2025-01 165.21 266.96    601.05  236.0 415.06 120.07 152.15 165.99  267.3   
2025-02 171.85 257.89    582.44 241.84 396.99 124.92 165.02 173.84 264.65   
2025-03  168.1 237.36    546.87 222.13 375.39 108.38 165.84 170.42  245.3   
2025-04 159.95 238.43    541.52  212.5 395.26 108.92 156.31 162.57 244.62   
2025-05 161.85 264.71    589.28 211.25 454.27  135.4 151.33 163.28 267.56   

                  Adj Close                                               \
        Benchmark      AAPL   MSFT   NVDA    JNJ     PG    JPM Benchmark   
Date                                                                       
2020-01    321.73     74.84  162.2   5.89 128.56 109.45 113.99    297.39   
2020-02    296.26     66.25 154.79   6.73 116.87  99.45  100.0    273.84   
2020-03    257.75     61.63 150.68   6.57 113.96  96.61  77.54    239.65   
2020-04    290.48     71.21 171.22   7.28  130.4 104.22  83.33    270.08   
2020-05    304.32     77.26 175.57   8.84 130.17 102.49  84.68    282.95   
...           ...       ...    ...    ...    ...    ...    ...       ...   
2025-01    601.82    235.43 413.47 120.06 150.94 164.96 265.66    600.02   
2025-02    59

In [ ]:
# This code creates a subset with the adjusted close for the stocks
stock_data = historical_data['Adj Close']
stock_data

,AAPL,MSFT,NVDA,JNJ,PG,JPM,Benchmark
Date,,,,,,,
2020-01,74.84,162.2,5.89,128.56,109.45,113.99,297.39
2020-02,66.25,154.79,6.73,116.87,99.45,100.0,273.84
2020-03,61.63,150.68,6.57,113.96,96.61,77.54,239.65
2020-04,71.21,171.22,7.28,130.4,104.22,83.33,270.08
2020-05,77.26,175.57,8.84,130.17,102.49,84.68,282.95
...,...,...,...,...,...,...,...
2025-01,235.43,413.47,120.06,150.94,164.96,265.66,600.02
2025-02,241.52,396.26,124.91,165.02,172.76,263.03,592.4
2025-03,221.84,374.7,108.38,165.84,169.36,243.8,559.39
